# Brain Tumor Detection

### For Google Colab Users

In [ ]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

### Init

In [1]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps, present_masks, calc_iou
from data import DataGenerator, SegmentationGenerator
from train import get_best_models_dict

# Show resources
!nvidia-smi
# Show possible models
print("Possible models:")
models_dict = get_best_models_dict()
for model in sorted(models_dict):
    print("Model {model} {spaces}-> accuracy {acc}".format(spaces=" "*(25-len(model)), model=model, acc=models_dict[model]))

wandb: Currently logged in as: ubrats (use `wandb login --relogin` to force relogin)


Wed Jul 28 17:43:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:B1:00.0 Off |                  Off |
| 33%   29C    P8    28W / 260W |      0MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Set Configs

In [2]:
default_config = {
    "model_name": "vgg19_seg",
    "input_size": (256,256), 
    "occlusion_size": (30,30),
    
    "epochs": 0,
  
    #"data_train_class": DataGenerator,         "data_train_path": os.path.join(".", "Data", "Training"),
    #"data_test_class": DataGenerator,          "data_test_path":  os.path.join(".", "Data", "Testing"),
    "data_train_class": SegmentationGenerator, "data_train_path": os.path.join(".", "SegmentationData", "train"), 
    "data_test_class": SegmentationGenerator,  "data_test_path":  os.path.join(".", "SegmentationData", "test-compact"),

    "gt_threshold": 0,
    "heatmap_threshold" : 0.575,

    "heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]],
    "ref_heat_layer": ['classifier', -1],
    "mri_type": "T1c", 
    "train_batch_size": 100,
    "lr": 1e-4,
    "heatmap_batch_size": 256,
    "smallest_contour_len" : 30,
    "shuffle_data": True,
    "device": "cuda",
    "attributes_to_log": ["model_name", "occlusion_size", "heatmap_threshold"]
}

sweep_config = {
    'name' : 'my-sweep',
    'method' : 'random',
    'parameters' : {
        'occlusion_size' : {'values' : [(10,10), (20,20), (30,30), (40,40), (50,50), (60,60)]},
        'gt_threshold' : {'values' : [0, 1, 2]},
        'heatmap_threshold' : {
            'distribution' : 'uniform', 
            'min': 0.4, 
            'max': 0.9
        }
    },
    'metric': {
        'name': 'avg_iou',
        'goal' : 'maximize'
    }
}

def agent():
    with wandb.init(project="BrainTumorDetection", config=default_config):
        config = wandb.config
        experiment = Experiment(config)
        iou = experiment.run()


### Run Single Experiment

In [14]:
agent()

loading vgg19_seg model...
loading best model state from: ./Models/vgg19_seg.pth


0it [00:00, ?it/s]

loading best model state from: ./Models/vgg19_seg.pth


Model's accuracy: 0.9
Creating heatmap...


100%|██████████| 256/256 [02:46<00:00,  1.54it/s]


...

100%|██████████| 256/256 [02:47<00:00,  1.53it/s]


Creating heatmap...


100%|██████████| 256/256 [02:47<00:00,  1.53it/s]


Test/accuracy/no,1.0
_runtime,1734
_timestamp,1627485951
_step,64
Test/accuracy/yes,0.8
Test/accuracy,0.9
#0/IOU,0.0
#1/IOU,0.42241
#2/IOU,0.0895
#3/IOU,0.3893
#4/IOU,0.41891


Test/accuracy/no,▁
_runtime,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test/accuracy/yes,▁
Test/accuracy,▁
#0/IOU,▁
#1/IOU,▁
#2/IOU,▁
#3/IOU,▁
#4/IOU,▁


### Sweep over many Experiments

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity='UBRATS', project='Unsupervised-BRATS')
print("Sweep ID:", sweep_id)
count = 100
wandb.agent(sweep_id, function=agent, count=count)

### Debug

#### plot funcs

In [6]:
import numpy as np

def move2cpu(d):
    """Move data from gpu to cpu"""
    return d.detach().cpu().float().numpy()

def tensor2im(im_t):
    """Copy the tensor to the cpu & convert to range [0,255]"""
    im_t = move2cpu(im_t)
    if len(im_t.shape) == 3:
        im_t = np.transpose(im_t, (1, 2, 0))
    im_np = np.clip(np.round(im_t * 255.0), 0, 255)
    return im_np.astype(np.uint8)

def im2tensor(im_np):
    """Copy the image to the gpu & converts to range [-1,1]"""
    im_np = im_np / 255.0 if im_np.dtype == 'uint8' else im_np
    return torch.FloatTensor(np.transpose(im_np, (2, 0, 1)) * 2.0 - 1.0).unsqueeze(0).cuda()

def plot(image, title=None):
    if type(image) != np.ndarray:
        image = tensor2im(image)
    
    plt.figure()
    if len(image.shape) == 2:
        plt.imshow(image, cmap="gray")
    else:
        plt.imshow(image)
    
    if title is not None: plt.title(title)
    plt.show()


#### play

In [3]:
from matplotlib import pyplot as plt

default_config["data_train_class"] = "data.SegmentationGenerator"
default_config["data_test_class"] = "data.SegmentationGenerator"

experiment = Experiment(default_config)

loading vgg19_seg model...
loading best model state from: ./Models/vgg19_seg.pth
